In [0]:
Data_path  = "/content/drive/My Drive/Location-Analytics/"

In [0]:
import json
import pandas as pd
from tqdm import tqdm

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
data = None
with open(Data_path+'small_data.json','r') as F:
    data = json.load(F)

In [0]:
type(data)

dict

In [0]:
data['data']

[{'accuracy': 1000,
  'activity': [{'activity': [{'confidence': 89, 'type': 'IN_VEHICLE'},
     {'confidence': 89, 'type': 'IN_ROAD_VEHICLE'},
     {'confidence': 4, 'type': 'STILL'},
     {'confidence': 3, 'type': 'IN_RAIL_VEHICLE'},
     {'confidence': 2, 'type': 'UNKNOWN'},
     {'confidence': 1, 'type': 'ON_FOOT'},
     {'confidence': 1, 'type': 'WALKING'},
     {'confidence': 0, 'type': 'ON_BICYCLE'}],
    'timestampMs': '1516589383476'}],
  'latitudeE7': 295642474,
  'longitudeE7': -985932226,
  'timestampMs': '1516589350178'},
 {'accuracy': 205,
  'activity': [{'activity': [{'confidence': 27, 'type': 'STILL'},
     {'confidence': 22, 'type': 'UNKNOWN'},
     {'confidence': 14, 'type': 'IN_VEHICLE'},
     {'confidence': 14, 'type': 'ON_FOOT'},
     {'confidence': 14, 'type': 'WALKING'},
     {'confidence': 14, 'type': 'IN_ROAD_VEHICLE'},
     {'confidence': 14, 'type': 'IN_RAIL_VEHICLE'},
     {'confidence': 6, 'type': 'ON_BICYCLE'},
     {'confidence': 5, 'type': 'RUNNING'}],
  

In [0]:
locations = data['data']

colunms to pick
1. timestamp(both)
2. high confidence activity
3. lat 
4. long


In [0]:
locations[10]

{'accuracy': 24,
 'activity': [{'activity': [{'confidence': 28, 'type': 'STILL'},
    {'confidence': 25, 'type': 'UNKNOWN'},
    {'confidence': 16, 'type': 'ON_FOOT'},
    {'confidence': 16, 'type': 'WALKING'},
    {'confidence': 10, 'type': 'IN_VEHICLE'},
    {'confidence': 10, 'type': 'IN_RAIL_VEHICLE'},
    {'confidence': 8, 'type': 'ON_BICYCLE'},
    {'confidence': 7, 'type': 'IN_ROAD_VEHICLE'},
    {'confidence': 6, 'type': 'RUNNING'}],
   'timestampMs': '1516588150153'}],
 'altitude': 272,
 'latitudeE7': 295641172,
 'longitudeE7': -985962843,
 'timestampMs': '1516588112874',
 'verticalAccuracy': 2}

In [0]:
class GoogleMapData:
    def __init__(self,accuracy,activity,latE7,longE7,timeStampMs):
        self.accuracy = accuracy
        self.activity = activity[0]['activity']
        self.latE7 = latE7
        self.longE7 = longE7
        self.timeStampMs_activity = activity[0]['timestampMs']
        self.timeStampMs = timeStampMs
    
    def get_accuracy(self):
        return self.accuracy
    
    def get_activity_list(self):
        return self.activity
    
    def get_latE7(self):
        return self.latE7
    
    def get_longE7(self):
        return self.longE7
    
    def get_timeStampMs_activity(self):
        return self.timeStampMs_activity
    
    def get_timeStampMs(self):
        return self.timeStampMs

    def get_highest_activity(self):
        high_acc_activity = self.activity[0]
        for ac in self.activity:
            if ac['confidence'] > high_acc_activity['confidence']:
                high_acc_activity = ac 
        return high_acc_activity


In [0]:
locations[0]

{'accuracy': 1000,
 'activity': [{'activity': [{'confidence': 89, 'type': 'IN_VEHICLE'},
    {'confidence': 89, 'type': 'IN_ROAD_VEHICLE'},
    {'confidence': 4, 'type': 'STILL'},
    {'confidence': 3, 'type': 'IN_RAIL_VEHICLE'},
    {'confidence': 2, 'type': 'UNKNOWN'},
    {'confidence': 1, 'type': 'ON_FOOT'},
    {'confidence': 1, 'type': 'WALKING'},
    {'confidence': 0, 'type': 'ON_BICYCLE'}],
   'timestampMs': '1516589383476'}],
 'latitudeE7': 295642474,
 'longitudeE7': -985932226,
 'timestampMs': '1516589350178'}

In [0]:
googleMal_Objs = []

for d in tqdm(locations,total = len(locations),unit = " record"):
    keys = list(d.keys())
    if 'accuracy' in keys and 'activity' in keys and 'latitudeE7' in keys and 'longitudeE7' in keys and 'timestampMs' in keys:
        g = GoogleMapData(d['accuracy'],d['activity'],d['latitudeE7'],d['longitudeE7'],d['timestampMs'])
        googleMal_Objs.append(g)

100%|██████████| 31891/31891 [00:00<00:00, 511645.66 record/s]


In [0]:
data_tuple = []

for g_data in googleMal_Objs:
    data_tuple.append((g_data.get_accuracy(),g_data.get_latE7(),g_data.get_longE7(),g_data.get_timeStampMs_activity(),
                       g_data.get_timeStampMs(),g_data.get_highest_activity()['confidence'],g_data.get_highest_activity()['type']))

In [0]:
df_columns = ['accuracy','latitudeE7','longitudeE7','timestampMs_activity','timestampMs','confidence','type']
df_data = pd.DataFrame(data = data_tuple,columns = df_columns)

In [0]:
df_data.head()

,accuracy,latitudeE7,longitudeE7,timestampMs_activity,timestampMs,confidence,type
0,1000,295642474,-985932226,1516589383476,1516589350178,89,IN_VEHICLE
1,205,295641414,-985927649,1516589258721,1516589230143,27,STILL
2,85,295641414,-985927649,1516589162481,1516589150162,100,TILTING
3,79,295643410,-985956502,1516589033330,1516589020860,83,ON_FOOT
4,28,295642341,-985957128,1516588870099,1516588900882,38,STILL


In [0]:
# if ( latitudeE7 > 900000000 ) then latitudeE7 = latitudeE7 - 4294967296
# if ( longitudeE7 > 1800000000) then longitudeE7 = longitudeE7 - 4294967296

In [0]:
def correct_error_lat(lat):
    if lat > 900000000:
        return lat - 4294967296
    else:
        return lat 

def correct_error_long(long_):
    if long_ > 1800000000:
        return long_ - 4294967296
    else:
        return long_



In [0]:
df_data['latitudeE7'] = df_data['latitudeE7'].apply(correct_error_lat)
df_data['longitudeE7'] = df_data['longitudeE7'].apply(correct_error_long)

In [0]:
from datetime import datetime

In [56]:
datetime.fromtimestamp(1515882991000000/1000.0)

ValueError: ignored

In [0]:
df_data['timestampMs'] = df_data['timestampMs'].apply(lambda x : datetime.fromtimestamp(int(x)/1000.0))
df_data['timestampMs_activity'] = df_data['timestampMs_activity'].apply(lambda x: datetime.fromtimestamp(int(x)/1000.0))

In [0]:
df_data.head()

,accuracy,latitudeE7,longitudeE7,timestampMs_activity,timestampMs,confidence,type
0,1000,295642474,-985932226,2018-01-22 02:49:43.476,2018-01-22 02:49:10.178,89,IN_VEHICLE
1,205,295641414,-985927649,2018-01-22 02:47:38.721,2018-01-22 02:47:10.143,27,STILL
2,85,295641414,-985927649,2018-01-22 02:46:02.481,2018-01-22 02:45:50.162,100,TILTING
3,79,295643410,-985956502,2018-01-22 02:43:53.330,2018-01-22 02:43:40.860,83,ON_FOOT
4,28,295642341,-985957128,2018-01-22 02:41:10.099,2018-01-22 02:41:40.882,38,STILL


In [0]:
df_data['lat'] = df_data['latitudeE7'] / 1e7
df_data['long'] = df_data['longitudeE7'] / 1e7

In [58]:
df_data.sort_values('timestampMs_activity',ascending=False).head()

,accuracy,latitudeE7,longitudeE7,timestampMs_activity,timestampMs,confidence,type,lat,long
0,1000,295642474,-985932226,2018-01-22 02:49:43.476,2018-01-22 02:49:10.178,89,IN_VEHICLE,29.564247,-98.593223
1,205,295641414,-985927649,2018-01-22 02:47:38.721,2018-01-22 02:47:10.143,27,STILL,29.564141,-98.592765
2,85,295641414,-985927649,2018-01-22 02:46:02.481,2018-01-22 02:45:50.162,100,TILTING,29.564141,-98.592765
3,79,295643410,-985956502,2018-01-22 02:43:53.330,2018-01-22 02:43:40.860,83,ON_FOOT,29.564341,-98.595650
4,28,295642341,-985957128,2018-01-22 02:41:10.099,2018-01-22 02:41:40.882,38,STILL,29.564234,-98.595713


In [0]:
df_sample = df_data.tail(100)

In [57]:
df_sample.sort_values('timestampMs_activity',ascending=False).head()

,accuracy,latitudeE7,longitudeE7,timestampMs_activity,timestampMs,confidence,type,lat,long
13858,39,129354174,775922181,2016-01-08 08:55:37.306,2016-01-08 08:55:32.262,58,UNKNOWN,12.935417,77.592218
13859,28,129354457,775922119,2016-01-08 08:54:40.429,2016-01-08 08:54:31.225,65,UNKNOWN,12.935446,77.592212
13860,31,129354182,775922532,2016-01-08 08:53:58.458,2016-01-08 08:53:51.468,100,TILTING,12.935418,77.592253
13861,30,129354296,775922585,2016-01-08 08:52:47.017,2016-01-08 08:52:49.208,42,UNKNOWN,12.935430,77.592258
13862,30,129354550,775922259,2016-01-08 08:52:03.221,2016-01-08 08:52:09.033,100,TILTING,12.935455,77.592226


In [0]:
import folium
print(folium.__version__)

0.8.3


In [0]:
import requests

In [0]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
vis1 = json.loads(requests.get(f'{url}/vis1.json').text)
vis2 = json.loads(requests.get(f'{url}/vis2.json').text)
vis3 = json.loads(requests.get(f'{url}/vis3.json').text)

In [0]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=[12.935074,77.591746],
    zoom_start=10,
    tiles='OpenStreetMap'
)

for lat,long,type_ in zip(df_sample['lat'],df_sample['long'],df_sample['type']):
    folium.Marker(
    location=[lat,long],
    popup=folium.Popup(max_width=450).add_child(
        folium.Vega(vis1, width=450, height=250))
    ).add_to(m)


In [0]:
m

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import pandas as pd

import plotly.express as px

fig = px.scatter_mapbox(df_data, lat="lat", lon="long", hover_name="type",
                        color_discrete_sequence=["red"], zoom=4, height=1000,width = 1500,hover_data = ['timestampMs','type'])
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [0]:

fig = px.line_mapbox(df_data, lat="lat", lon="long", color="type", zoom=5, height=1000,width = 1500)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = 16,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()